In [ ]:
"""
Created on Fri Jan 27 16:14 2023

This script is to check the input T and S profiles from Smith

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import basal_melt_param.melt_functions as meltf
from basal_melt_param.constants import *
import basal_melt_param.T_S_profile_functions as tspf
import seaborn as sns
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import cmocean
import glob

In [ ]:
%matplotlib qt5

READ DATA

In [ ]:
plot_path = '/bettik/burgardc/PLOTS/NN_plots/input_vars/'
outputpath_simple_all = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/SIMPLE/nemo_5km_06161821/'


In [ ]:
isf_dim = [10,11,12,13,18,22,23,25,30,31,33,39,40,42,43,44,45,47,51,55,58,61,65,66,69,70,71,73,75]

In [ ]:
inputpath_profiles = '/bettik/burgardc/DATA/NN_PARAM/interim/T_S_PROF/SMITH_bi646/'
yy_list = range(1970,1970+35)
prof_list = []
for yy in yy_list:
    file_TS_yy = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_'+str(yy)+'.nc')
    prof_list.append(file_TS_yy)
    
file_TS = xr.concat(prof_list, dim='time').assign_coords({'nemo_run': 'bi646'})
file_TS_mean_bi646 = file_TS.mean('time')
file_TS_std_bi646 = file_TS.std('time')

In [ ]:
inputpath_profiles = '/bettik/burgardc/DATA/NN_PARAM/interim/T_S_PROF/SMITH_bf663/'
yy_list = range(1970,1970+40)
prof_list = []
for yy in yy_list:
    file_TS_yy = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_'+str(yy)+'.nc')
    prof_list.append(file_TS_yy)
    
file_TS = xr.concat(prof_list, dim='time').assign_coords({'nemo_run': 'bf663'})
file_TS_mean_bf663 = file_TS.mean('time')
file_TS_std_bf663 = file_TS.std('time')

In [ ]:
inputpath_mask = '/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_bf663/'
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy54.nc')
file_isf = file_isf_orig.sel(Nisf = isf_dim).squeeze().drop('time')

In [ ]:
file_TS_mean_all = xr.concat([file_TS_mean_bf663, file_TS_mean_bi646], dim='nemo_run')
file_TS_std_all = xr.concat([file_TS_std_bf663, file_TS_std_bi646], dim='nemo_run')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','brown','magenta','yellow','cyan','blue','magenta']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0
for rr,reg in enumerate(regions):
    
    subset_isf = file_isf.Nisf.where(file_isf['region']==reg,drop=True)
    
    for kisf in tqdm(subset_isf.Nisf):

        mean_data = file_TS_mean_all['theta_ocean'].sel(Nisf=kisf)
        std_data = file_TS_std_all['theta_ocean'].sel(Nisf=kisf)
        
        ax[i] = f.add_subplot(6,6,i+1)
        
        for n, nrun in enumerate(mean_data.nemo_run):
            
            mean_run = mean_data.sel(nemo_run=nrun)
            std_run = std_data.sel(nemo_run=nrun)
            
            leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
            plt.fill_betweenx(-1*file_TS_mean_all.depth, mean_run - std_run, mean_run + std_run, -1*file_TS_mean_all.depth, alpha=0.2, color=colors[n])
            leg_hdl.append(leg)
            
        
        ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
        ax[i].set_xlim(-2.2,2.5)
        
        i = i+1
        

f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','brown','magenta','yellow','cyan','blue','magenta']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0
for rr,reg in enumerate(regions):
    
    subset_isf = file_isf.Nisf.where(file_isf['region']==reg,drop=True)
    
    for kisf in tqdm(subset_isf.Nisf):

        mean_data = file_TS_mean_all['salinity_ocean'].sel(Nisf=kisf)
        std_data = file_TS_std_all['salinity_ocean'].sel(Nisf=kisf)
        
        ax[i] = f.add_subplot(6,6,i+1)
        
        for n, nrun in enumerate(mean_data.nemo_run):
            
            mean_run = mean_data.sel(nemo_run=nrun)
            std_run = std_data.sel(nemo_run=nrun)
            
            leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
            plt.fill_betweenx(-1*file_TS_mean_all.depth, mean_run - std_run, mean_run + std_run, -1*file_TS_mean_all.depth, alpha=0.2, color=colors[n])
            leg_hdl.append(leg)
            
        
        ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
        ax[i].set_xlim(31,35)
        
        i = i+1
        

f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()